In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### UnderStanding the The Envorment in Open_ai_gym

In [5]:
env=gym.make('CartPole-v1')

In [7]:
episodes=10

for episode in range(1,episodes+1):
    state=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action=env.action_space.sample()
        n_state,reward,done,info=env.step(action)
        score+= reward
    print(episode ,score)
env.close()

1 16.0
2 21.0
3 37.0
4 9.0
5 21.0
6 18.0
7 29.0
8 11.0
9 23.0
10 13.0


In [8]:
episodes=5 
for i in range(1,episodes+1):
    print(episodes)

5
5
5
5
5


In [9]:
env.reset()

array([-0.04871323, -0.01589516, -0.01716098,  0.03014504], dtype=float32)

In [10]:
# in this envorment there is two Discrete Value 
# 1==== Push Cart  to the Left 
# 0======Push Cart to the Right
env.action_space

Discrete(2)

In [11]:
#the first number represents cart Position
# the second number represents Cart Velocity
#the third number represent  the Pole angle which is radinan
# the Forth number  represent the POle angueler velocity

env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [12]:
for i in range(5):
    print(env.step(1))
        

(array([-0.04903113,  0.17946863, -0.01655808, -0.26790258], dtype=float32), 1.0, False, {})
(array([-0.04544176,  0.3748229 , -0.02191613, -0.5657617 ], dtype=float32), 1.0, False, {})
(array([-0.0379453 ,  0.5702454 , -0.03323137, -0.8652678 ], dtype=float32), 1.0, False, {})
(array([-0.0265404 ,  0.7658035 , -0.05053673, -1.1682112 ], dtype=float32), 1.0, False, {})
(array([-0.01122433,  0.96154517, -0.07390095, -1.4763004 ], dtype=float32), 1.0, False, {})


In [13]:
# the  Reward is 1 for every step taken ,including the Termination step

reward

1.0

## Traning the Model


In [14]:
log_path=os.path.join('traning','logs')

In [15]:
log_path

'traning\\logs'

In [16]:
env=gym.make('CartPole-v1')
env=DummyVecEnv([lambda:env])
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [33]:
model.learn(total_timesteps=200000)

Logging to traning\logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 250  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 369         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008776711 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00275    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.31        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 54.9        |
-----------------------------------------
----

### Saving the Model

In [35]:
PPO_path=os.path.join('traning','saved_model','PPO_model_Cartpole')

In [37]:
model.save(PPO_path)


# to delete model  syntax=   del model

In [38]:
model=PPO.load(PPO_path,env=env)

In [43]:
model.learn(total_timesteps=1000)

Logging to traning\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1527 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


###  Evaluation

In [44]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

C:\Users\skidrow_pc\Anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(500.0, 0.0)

In [45]:
env.close()

### Testing the Model

In [49]:
episodes=10

for episode in range(1,episodes+1):
    obs=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action,_=model.predict(obs) #  now the using the traned modle to prdict
        obs,reward,done,info=env.step(action)
        score+= reward
    print(episode ,score)
env.close()

1 [500.]
2 [500.]
3 [500.]
4 [500.]
5 [500.]
6 [500.]
7 [500.]
8 [500.]
9 [500.]
10 [500.]


In [50]:
env.close()

In [52]:
obs=env.reset()

In [53]:
model.predict(obs)

(array([1], dtype=int64), None)

In [60]:
model.action_space.sample()

1

In [64]:
env.step(action)

(array([[ 0.02860805,  0.827162  , -0.04453079, -1.1654762 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

### Viewing the Logs in the Tensorflow

In [65]:
traning_log_path=os.path.join(log_path,'PPO_2')

In [66]:
traning_log_path

'traning\\logs\\PPO_2'

In [ ]:
!tensorboard --logdir=[traning_log_path]

### Applying the Callbacks feature
- you can leverage callback fuctions as a part of the Stable baseline  to log out data or save the model under cetain condition


In [1]:
from stable_baselines3.common.callbacks import EvalCallback,StopTrainingOnRewardThreshold

In [2]:
stop_calback= StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)

In [18]:
save_path=os.path.join('traning','saved_model')

In [19]:
eval_callback=EvalCallback(env,
                          callback_on_new_best=stop_calback,
                          eval_freq=10000,
                          best_model_save_path=save_path,
                          verbose=1)

In [20]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [21]:
model.learn(total_timesteps=10000,callback=eval_callback)

Logging to traning\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 319  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 443         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007826226 |
|    clip_fraction        | 0.0806      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.0011      |
|    learning_rate        | 0.0003      |
|    loss                 | 6.36        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0114     |
|    value_loss           | 52.8        |
-----------------------------------------
----

C:\Users\skidrow_pc\Anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=420.00 +/- 97.98
Episode length: 420.00 +/- 97.98
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 420         |
|    mean_reward          | 420         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009802561 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.611      |
|    explained_variance   | 0.332       |
|    learning_rate        | 0.0003      |
|    loss                 | 27.9        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0203     |
|    value_loss           | 65.7        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 420.00  is above the threshold 200


### Changing the Polices
- means to change the Nural network Architecture

In [60]:
net_arch=dict(v=[128,128,128,128])

In [64]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path,policy_kwargs=('net_arch',dict(v=[128,128,128,128])))

Using cpu device


TypeError: stable_baselines3.common.policies.ActorCriticPolicy() argument after ** must be a mapping, not tuple

dict_keys(['v'])

### Using the Different Model 

In [74]:
from stable_baselines3 import DQN,A2C

In [72]:
model=DQN('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [73]:
model.learn(total_timesteps=1000)

Logging to traning\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.278    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2532     |
|    time_elapsed     | 0        |
|    total_timesteps  | 76       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3080     |
|    time_elapsed     | 0        |
|    total_timesteps  | 154      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 3571     |
|    time_elapsed     | 0        |
|    total_timesteps  | 300      |
----------------------------------
-------------------------

In [75]:
model=A2C('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [76]:
model.learn(total_timesteps=1000)

Logging to traning\logs\A2C_1
------------------------------------
| time/                 |          |
|    fps                | 703      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.652   |
|    explained_variance | -0.212   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.47     |
|    value_loss         | 10.5     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 691      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.636   |
|    explained_variance | 0.216    |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 1.02     |
|    val